In [1]:
#I try to predict values not used in a model to predict results.
#Internet is plenty of models using X_test to predict.
#However, I havent seen a lot of models using data outside the model
#Thank you

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/EURUSD"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

/root/anaconda2/envs/mla36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


EURUSD1440.csv
EURUSD_2017.csv



In [3]:
seq_len = 25
d = 0.5 #dropout
shape = [4, seq_len, 1] # feature, window, output
neurons = [128, 128, 32, 1]
epochs = 75
dropout=0.6
stock_name='../input/EURUSD/EURUSD1440.csv'

In [4]:
def get_stock_data(stock_name,inicio='2012',final='2016', normalize=True):
    
    df=pd.read_csv(stock_name,names=['Date','Open','High','Low','Close'],usecols=[0,2,3,4,5],index_col=[0])
    df=df.loc[inicio:final]

    df.reset_index(inplace=True,drop=True)
    if normalize:        
        min_max_scaler = preprocessing.MinMaxScaler()
        df['Open'] = min_max_scaler.fit_transform(df.Open.values.reshape(-1,1))
        df['High'] = min_max_scaler.fit_transform(df.High.values.reshape(-1,1))
        df['Low'] = min_max_scaler.fit_transform(df.Low.values.reshape(-1,1))
        df['Close'] = min_max_scaler.fit_transform(df['Close'].values.reshape(-1,1))
    return df

In [5]:
# Formulas to make LSTM model
def load_data(stock, seq_len,train_split=0.9):
    amount_of_features = len(stock.columns)
    data = stock.as_matrix() 
    sequence_length = seq_len + 1 # index starting from 0 esto se hace ya que el indice empieza por 0 entonces quedaria

    result = []
    for index in range(len(data) - sequence_length): # maxmimum date = lastest date - sequence length
        result.append(data[index: index + sequence_length]) # index : index + 22days 
    result = np.array(result)
    row = round(train_split * result.shape[0]) # 90% split
    
    train = result[:int(row), :] # 90% date
    X_train = train[:, :-1] # all data until day m
    y_train = train[:, -1][:,-1] 
    


    
    X_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:,-1] 

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], amount_of_features))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], amount_of_features))  

    return [X_train, y_train, X_test, y_test]


def build_model2(layers, neurons, d):
    model = Sequential()
    
    model.add(LSTM(neurons[0], input_shape=(layers[1], layers[0]), return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(neurons[1], input_shape=(layers[1], layers[0]), return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(neurons[2],kernel_initializer="uniform",activation='relu'))        
    model.add(Dense(neurons[3],kernel_initializer="uniform",activation='linear'))
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [6]:
df = get_stock_data(stock_name,inicio='2013',final='2016', normalize=True)
df_denormalize = get_stock_data(stock_name,inicio='2013',final='2016',normalize=False)

In [ ]:
X_train, y_train, X_test, y_test = load_data(df, seq_len,train_split=0.9)

model3 = build_model2(shape, neurons, d)

X_train, y_train, X_test, y_test = load_data(df, seq_len,train_split=0.9)

model3.fit(X_train,y_train,batch_size=512,epochs=epochs,validation_split=0.1,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 128)           68096     
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 203,841
Trainable params: 203,841
Non-trainable params: 0
_________________________________________________________________
Trai

610/610 [==============================] - 2s 3ms/step - loss: 0.0087 - acc: 0.0033 - val_loss: 0.0017 - val_acc: 0.0000e+00
Epoch 53/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0083 - acc: 0.0033 - val_loss: 0.0017 - val_acc: 0.0000e+00
Epoch 54/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0092 - acc: 0.0033 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 55/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0087 - acc: 0.0033 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 56/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0077 - acc: 0.0033 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 57/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0081 - acc: 0.0033 - val_loss: 0.0015 - val_acc: 0.0000e+00
Epoch 58/75
610/610 [==============================] - 2s 3ms/step - loss: 0.0080 - acc: 0.0033 - val_loss: 0.0015 - val_acc: 0.0000e+00
Epoch 59/75
610/610 [==============================] 

In [ ]:
# Making predictions with new data not used in the model

def datos_predecir(file='../input/EURUSD/EURUSD_2017.csv',normalise=True):
    df=pd.read_csv(file,usecols=[1,2,3,4,5],index_col=[0])
    df.reset_index(inplace=True,drop=True)  
    if normalise==True:
        min_max_scaler = preprocessing.MinMaxScaler()
        df['Open'] = min_max_scaler.fit_transform(df.Open.values.reshape(-1,1))
        df['High'] = min_max_scaler.fit_transform(df.High.values.reshape(-1,1))
        df['Low'] = min_max_scaler.fit_transform(df.Low.values.reshape(-1,1))
        df['Close'] = min_max_scaler.fit_transform(df['Close'].values.reshape(-1,1))
    return df

In [ ]:
predict_data_norm=datos_predecir()
predict_data_denorm=datos_predecir(normalise=False)

In [ ]:
predict_data_length=[]
data=predict_data_norm.as_matrix()
for index in range(len(data)-seq_len):
    predict_data_length.append(data[index: seq_len + index])

datos_predecir=np.array(predict_data_length)

prediccion=model3.predict(datos_predecir)
plt.plot(prediccion)
plt.plot(predict_data_norm['Close'])
plt.show()